# PyBullet을 이용한 진자(Pendulum) 강화학습 환경

MATLAB의 진자 예제와 유사한 강화학습 환경을 PyBullet으로 구성

- **상태**: `[cos(theta), sin(theta), theta_dot]`
- **행동**: 진자 관절에 가하는 토크 (-2.0 ~ 2.0)
- **보상**: 진자가 위로 설수록, 각속도와 토크가 작을수록 높은 보상 획득.

### 1단계: URDF 파일 생성 (Code)

Jupyter Notebook의 매직 명령어 %%writefile을 사용하면 현재 디렉토리에 바로 파일을 생성할 수 있어 편리하다고 함!

In [ ]:
%%writefile models/pendulum.urdf
<?xml version="1.0"?>
<robot name="pendulum">
  <link name="base"/>

  <link name="pole">
    <inertial>
      <mass value="1.0"/>
      <inertia ixx="0.083" ixy="0.0" ixz="0.0" iyy="0.083" iyz="0.0" izz="0.005"/>
    </inertial>
    <visual>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
      <material name="red">
          <color rgba="1 0.2 0.2 1"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
    </collision>
  </link>

  <joint name="hinge" type="revolute">
    <parent link="base"/>
    <child link="pole"/>
    <axis xyz="0 1 0"/>
    <limit effort="3" lower="-3.1415" upper="3.1415" velocity="15"/>
    <origin rpy="0 0 0" xyz="0 0 0.5"/>
  </joint>

</robot>

### 2단계: 라이브러리 임포트 (Code)

환경 구성과 테스트에 필요한 라이브러리들을 불러오기

In [2]:
import gymnasium as gym
from gymnasium import spaces
import pybullet as p
import pybullet_data
import numpy as np
import cv2
import time
print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


pybullet build time: May 17 2025 21:05:07


## 3단계: 시뮬 구성하기
### 3-1 막대와 바닥과 중력 

In [4]:
URDF_PATH = "models/pendulum.urdf" # 실제 URDF 파일 경로

try:
    client = p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    
    robot_id = p.loadURDF(URDF_PATH)

    # 1. 전체 조인트 개수 확인
    num_joints = p.getNumJoints(robot_id)
    print(f"'{URDF_PATH}'에 총 {num_joints}개의 조인트가 있습니다.")
    print("-" * 40)

    # 2. 각 조인트의 정보 출력
    for i in range(num_joints):
        joint_info = p.getJointInfo(robot_id, i)
        
        joint_index = joint_info[0]
        joint_name = joint_info[1].decode('utf-8') # byte string을 일반 string으로 변환
        joint_type = joint_info[2]
        child_link_name = joint_info[12].decode('utf-8')
        
        print(f"  인덱스 (Index): {joint_index}")
        print(f"  이름 (Name): {joint_name}")
        print(f"  타입 (Type): {joint_type} (REVOLUTE=0, FIXED=4 등)")
        print(f"  연결된 링크 (child Link): {child_link_name}")
        print("-" * 40)

finally:
    if p.isConnected():
        p.disconnect()

'models/pendulum.urdf'에 총 3개의 조인트가 있습니다.
----------------------------------------
  인덱스 (Index): 0
  이름 (Name): fixed_base
  타입 (Type): 4 (REVOLUTE=0, FIXED=4 등)
  연결된 링크 (child Link): base
----------------------------------------
  인덱스 (Index): 1
  이름 (Name): hinge
  타입 (Type): 0 (REVOLUTE=0, FIXED=4 등)
  연결된 링크 (child Link): pole
----------------------------------------
  인덱스 (Index): 2
  이름 (Name): fixed_ball
  타입 (Type): 4 (REVOLUTE=0, FIXED=4 등)
  연결된 링크 (child Link): ball
----------------------------------------
5초 후 종료됩니다...


In [4]:
width, height = 640, 480
frames = []

p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.81)
urdf_path = "/models/stick.urdf"
print(urdf_path)

pendulum_id = p.loadURDF(urdf_path, basePosition=[0, 0, 1], useFixedBase=False)

for step in range(240*5):
    p.stepSimulation()

    img = p.getCameraImage(width, height, renderer=p.ER_TINY_RENDERER)
    rgb = np.array(img[2]).reshape((height, width, 4))[:,:,:3]
    frames.append(rgb)

    if step % 100 == 0:
        print(step)
        
video = cv2.VideoWriter('video/test.mp4', cv2.VideoWriter.fourcc(*'avc1'), 20.0, (width, height))
for frame in frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video.release()

p.disconnect()

/models/stick.urdf
0
100
200
300
400
500
600
700
800
900
1000
1100


이걸 돌리고 나면 GIF 파일로 확인할 수 있는 것처럼 막대가 낙하하고, 바닥에 부딪혀 넘어지는 것이 확인되어야 한다.
![stick_plane_gravity](docs/step_3_1.gif) 

### 3-2 고정되고, 회전하는 막대

In [21]:
width, height = 640, 480
frames = []

p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.81)
urdf_path = "/Users/seunghyunpyo/PycharmProjects/bullet_tutorial/models/pendulum.urdf"
print(urdf_path)

poleStartPos = [0,0,0]
poleStartOrientation = p.getQuaternionFromEuler([0,0,0])
pendulum_id = p.loadURDF(urdf_path, basePosition=poleStartPos, baseOrientation=poleStartOrientation, useFixedBase=True)

initial_angle = np.pi*(1/3)  # 새 URDF의 hinge 조인트는 첫 번째(유일한) 조인트이므로 인덱스는 0입니다.
p.resetJointState(
    bodyUniqueId=pendulum_id,
    jointIndex=1,
    targetValue=initial_angle
)
print(f"진자를 {np.rad2deg(initial_angle):.0f}도 위치에서 시작")

p.setJointMotorControl2(
        bodyUniqueId=pendulum_id,
        jointIndex=1,
        controlMode=p.VELOCITY_CONTROL,
        force=0
    )

for step in range(240*2):
    p.stepSimulation()

    img = p.getCameraImage(width, height, renderer=p.ER_TINY_RENDERER)
    rgb = np.array(img[2]).reshape((height, width, 4))[:,:,:3]
    frames.append(rgb)

    if step % 100 == 0:
        print(step)

video = cv2.VideoWriter('video/test.mp4', cv2.VideoWriter.fourcc(*'avc1'), 20.0, (width, height))
for frame in frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video.release()

p.disconnect()

/Users/seunghyunpyo/PycharmProjects/bullet_tutorial/models/pendulum.urdf
진자를 60도 위치에서 시작
0
100
200
300
400


### 4단계: 클래스로 PyBullet 진자 환경 정의하기
여기에 클래스 작성하기

### 5단계: 환경 테스트 - 클래스

정의된 환경을 실제로 생성하고, 무작위 행동을 주면서 어떻게 동작하는지 테스트
우선, 불러온 펜듈럼이 시뮬에서 제대로 돌아가는지나 보자. 